In [1]:
using LinearAlgebra: I, Matrix

struct Dual <: Number
    val::Real
    grad::Vector{Real}
end

function Dual(vars::Vector{<:Real})
    l = length(vars)
    id = Matrix(I, l, l)
    map(1:l) do j
        Dual(vars[j], id[:,j])
    end
end

Base.convert(::Type{Dual}, x::Real) = Dual(x, [zero(x)])
Base.promote_rule(::Type{Dual}, ::Type{<:Real}) = Dual

Base.:+(f::Dual, g::Dual) = Dual(f.val + g.val, f.grad .+ g.grad)
Base.:-(f::Dual, g::Dual) = Dual(f.val - g.val, f.grad .- g.grad)
Base.:*(f::Dual, g::Dual) = Dual(f.val * g.val, f.grad .* g.val .+ f.val * g.grad)
Base.:/(f::Dual, g::Dual) = Dual(f.val / g.val, (f.grad .* g.val .- f.val .* g.grad) ./ g.val^2)

Base.:-(f::Dual) = 0 - f

Base.sqrt(f::Dual) = Dual(sqrt(f.val), f.grad ./ sqrt(f.val))
Base.log(f::Dual) = Dual(log(f.val), f.grad ./ f.val)
Base.conj(f::Dual) = Dual(conj(f.val), conj.(f.grad))

In [2]:
f(x,y) = x^2 - y^2

f(Dual([-1,1])...)

Dual(0, Real[-2, -2])

In [3]:
entropy(λ) = -λ'log.(λ)

entropy(Dual([2/3, 2/9, 1/9]))

Dual(0.8486855577264172, Real[-0.5945348918918356, 0.5040773967762742, 1.1972245773362196])

In [9]:
uncert(f::Dual, σ::Vector{<:Real}) = (val = f.val, err = √sum((f.grad .* σ).^2))

uncert (generic function with 1 method)

In [10]:
p = [2/3, 2/9, 1/9]
σ = [0.1, 0.12, 0.08]
s = entropy(Dual(p))
uncert(s, σ)

(val = 0.8486855577264172, err = 0.12793392864918046)